In [6]:
import re
import MySQLdb
import bz2

def get_errors(cid, fullLog):
    try:
        socket = MySQLdb.connect("148.60.11.195", "script", "irmadb2019", "IrmaDB_prod")
        cursor = socket.cursor()

        query = "SELECT errlog_file FROM Compilations WHERE cid = %d" % cid
        cursor.execute(query)
        errlog_file = cursor.fetchone()
        
            
        if (errlog_file is None):
            print ("Unable to retrieve cid=", str(cid))
            return

        try:
            log = bz2.decompress(errlog_file[0]).decode('utf-8')
            # fatal error
            m1 = re.findall('.* error:.+', log)
            m2 = re.findall('. undefined reference.+', log)
            # '1:' corresponds to the error status I guess
            m3 = re.findall('.* 1:.+', log)
            # may not be useful: make errors
            m4 = re.findall('make.*: \*\*\*.+', log)
            return m1 + m2 + m3 + m4
        except Exception as e:
            print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
            exit(-1)
    
    except Exception as e:
        print(str(e),"\n" + "Unable to connect to database c = " + str(cid), file=sys.stderr)
        exit(-1)
        
    finally:
        cursor.close()
        socket.close()

#cids = [61083, 57231, 34513, 52365, 52941, 62788, 73627, 55836, 70534, 55064]
#cids = [35706, 57116, 33574, 56926, 44782, 78452, 37565, 65481, 77080, 60274]
cids = [31273]
for cid in cids:
    print('errors for ' + str(cid) + ': ' + str(get_errors(cid, False)))
    print('************')
        

errors for 31273: [": undefined reference to `v4l2_async_unregister_subdev'", ": undefined reference to `v4l2_subdev_init'", ": undefined reference to `v4l2_async_register_subdev'", ": undefined reference to `v4l2_subdev_link_validate'", 'make: *** [vmlinux] Error 1']
************


In [38]:
import re
import MySQLdb
import bz2
       
cids = []
try:
    socket = MySQLdb.connect("148.60.11.195", "script", "irmadb2019", "IrmaDB_prod")
    cursor = socket.cursor()
    #query = "SELECT cid FROM Compilations WHERE core_size=-1 and cid > 30000 and cid < 100000"
    query = "SELECT cid FROM Compilations WHERE core_size=-1 and cid > 30000 and cid < 33000"
    #query = "SELECT cid FROM Compilations WHERE cid in ()"
    #cid = 78276 
    cursor.execute(query)
    row = cursor.fetchone()
    while row is not None:
        cids.append(int(str(row).replace("(","").replace(",","").replace(")","")))
        row = cursor.fetchone()
    print('CIDS - ',cids)

    lista = []
    listoflist = []
    tamanho = 0

    for cid in cids:
       
        query = "SELECT errlog_file FROM Compilations WHERE cid = %d" % cid
        cursor.execute(query)
        errlog_file = cursor.fetchone()
        
        #print(cid)
        
        if (errlog_file is None):
           print ("Unable to retrieve cid=", str(cid))

        try:
           log = bz2.decompress(errlog_file[0]).decode('utf-8')
           # fatal error
           #m1 = re.findall('.* error:.+', log)
           m1 = re.findall('.* error:.+', log)
           #m2 = re.findall('.* undefined reference.+', log)
           m2 = re.findall('. undefined reference to `gen_pool.', log)
           # '1:' corresponds to the error status I guess
           #m3 = re.search('. undefined reference to \'v4l2.+', log)
           #m4 = ('. undefined reference to \'drm.+', log)
           m5 = re.findall('.* 1:.+', log)
           # may not be useful: make errors
           #m4 = re.findall('make.*: \*\*\*.+', log)
           log_error = str(m1) + str(m2) + str(m5)
        except Exception as e:
           print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
           exit(-1)
    
        ja_existe=False
        if tamanho==0:
           lista=[log_error,cid]
           listoflist.append(lista)
           tamanho=1
        else:
            for i in range(tamanho):
                if (listoflist[i][0]==log_error):
                    listoflist[i].append(cid)
                    ja_existe=True
            if ja_existe==False:
                lista=[log_error,cid]
                listoflist.append(lista)
                tamanho=tamanho+1

except Exception as e:
    print(str(e),"\n" + "Unable to connect to database c = " + str(cid), file=sys.stderr)
    exit(-1)    
finally:
    cursor.close()
    socket.close()

file_log = open('clusters.csv', 'w')

for i in range(tamanho):
    ncids=len(listoflist[i])
    #print(listoflist[i][0],';')
    reg = (str(listoflist[i][0])+';')
    for j in range(ncids-1):
        reg = (reg+str(listoflist[i][j+1])+';')
        #print('CIDS: ',listoflist[i][j+1])

    file_log.write(reg+'\n')

file_log.close()

CIDS -  [30699, 30700, 30704, 30717, 30718, 30722, 30726, 30730, 30731, 30737, 30806, 30811, 30821, 30837, 30849, 30879, 30886, 30891, 30901, 30904, 30917, 30927, 30959, 30961, 30962, 30964, 30971, 30989, 31012, 31025, 31041, 31070, 31080, 31098, 31102, 31106, 31119, 31124, 31158, 31177, 31188, 31199, 31200, 31203, 31205, 31213, 31232, 31237, 31260, 31262, 31268, 31273, 31281, 31298, 31299, 31302, 31316, 31318, 31327, 31334, 31335, 31336, 31341, 31356, 31364, 31385, 31390, 31393, 31399, 31411, 31413, 31416, 31420, 31427, 31445, 31459, 31461, 31492, 31493, 31536, 31545, 31560, 31567, 31600, 31623, 31628, 31658, 31687, 31717, 31738, 31741, 31745, 31766, 31786, 31799, 31802, 31803, 31816, 31843, 31870, 31877, 31883, 31899, 31905, 31909, 31935, 31947, 31986, 32006, 32031, 32034, 32050, 32054, 32083, 32120, 32126, 32157, 32161, 32196, 32197, 32207, 32209, 32214, 32234, 32236, 32240, 32266, 32284, 32290, 32292, 32305, 32318, 32326, 32378, 32394, 32401, 32407, 32415, 32419, 32430, 32446, 3245

In [7]:
import pandas as pd
import numpy as np

# download config_bdd30-100.pkl here 
# https://filesender.renater.fr/?s=download&token=556ae671-4546-493e-f96f-3cd3c1919724#

# TODO: excellent "test" set since 15K and 30K have not been subject to specialization
# pd.read_csv('config_bdd15000-20000.csv') 
# pd.read_csv('config_bdd20000-30000.csv') 
rawtuxdata = pd.read_pickle('config_bdd30-100.pkl')

basic_head = ["cid", "time", "date"] # "compile"
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

print("some configurations may have X86_32 (coz we have tested/tried some options and there are in the database) -- we removed them of course")
# we only keep X86_64 configurations
#rawtuxdata = rawtuxdata[rawtuxdata['X86_64'] == 'y'] ### TODO: I've impression it's not the most effective way (wrt memory) to filter 
if 'X86_64' in rawtuxdata.columns:
    print(rawtuxdata['X86_64'].describe())
    rawtuxdata.query("X86_64 == 'y'", inplace=True) # only keep them

some configurations may have X86_32 (coz we have tested/tried some options and there are in the database) -- we removed them of course
count     69459
unique        2
top           y
freq      69423
Name: X86_64, dtype: object


In [8]:
### TODO: ensure that the same configuration leads to the same compilation status (normally it's the case)
# remove entries with same configurations
print(str(len(rawtuxdata)) + " before the removal of some entries (those with same configurations)")
# tuxdata.drop_duplicates(subset=tuxdata.columns.difference(['vmlinux']), inplace=True)
rawtuxdata.drop_duplicates(subset=rawtuxdata.columns.difference(size_methods).difference(basic_head), inplace=True)
print(str(len(rawtuxdata)) + " after the removal of some entries (those with same configurations)")

69423 before the removal of some entries (those with same configurations)
69364 after the removal of some entries (those with same configurations)


In [9]:
cid_to_remove=74459 # compilation time is very low (2 sec)
cid_to_remove2=30698 # size is 0! 

rawtuxdata = rawtuxdata[rawtuxdata.cid != cid_to_remove]
rawtuxdata = rawtuxdata[rawtuxdata.cid != cid_to_remove2]

In [10]:
len(rawtuxdata)

69362

In [11]:
len(rawtuxdata.query("vmlinux == -1").index)

3507

In [12]:
def nbyes(row):
    return sum(row == "y")                    
rawtuxdata['nbyes'] = rawtuxdata.apply(nbyes, axis=1)
rawtuxdata.sort_values(by="nbyes")[:10]

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,nbyes
14299,74458,2018-07-05 16:07:21,28.7108,7317008,646608.0,2733176,501222,4722128.0,6808144,458568,...,n,y,n,n,n,n,n,n,n,240
16734,57051,2018-06-26 17:44:03,350.4990,28838328,6677456.0,8762488,6531115,10453968.0,12538448,6188913,...,n,y,y,n,n,n,n,n,n,668
4723,34765,2018-06-13 00:11:19,120.3810,64176376,3617744.0,5704408,3407623,7807952.0,9894056,3479187,...,n,y,n,n,n,n,n,n,n,684
20435,80598,2018-07-25 14:33:52,270.8170,14808224,2303440.0,4388304,2154587,6346192.0,8430504,2080144,...,n,y,y,n,n,n,n,n,n,708
14064,74219,2018-07-05 09:25:11,134.2110,13729408,3380176.0,5465552,3233544,7439312.0,9524136,3173721,...,m,y,n,n,n,n,n,m,n,710
13877,53941,2018-06-16 13:24:35,172.9770,25078328,2339792.0,4425168,2192433,6382544.0,8467368,2117897,...,n,y,n,y,n,n,n,n,n,713
1891,61891,2018-06-27 08:31:38,322.6870,65978424,2691024.0,4778824,2488385,6881232.0,8968480,2563987,...,n,y,n,n,n,n,n,n,n,715
20162,80325,2018-07-25 13:05:31,173.8940,12783848,2270160.0,4355536,2121304,6337488.0,8422312,2072046,...,m,y,n,n,n,n,n,m,n,718
1112,31115,2018-06-11 18:46:09,156.1740,73461752,3081168.0,5166552,2933658,7173072.0,9257896,2906856,...,m,y,n,n,n,n,n,m,n,730
14306,74465,2018-07-05 17:11:54,225.9910,48912544,6660048.0,8746104,6515085,10166224.0,12251728,5903503,...,n,y,y,n,n,m,n,n,n,730


In [8]:
import pandas as pd

errors = pd.read_csv('errors.csv')

rawtuxdata.query("vmlinux == -1", inplace=True) # only keep cids with kernel size == -1 
#print (len(rawtuxdata))

rawtuxdata['cluster'] = 'null'
num = len(rawtuxdata.columns) - 1 

i = 0
for row in rawtuxdata.iterrows():
    cid = rawtuxdata.iloc[i, 0]
    log = errors.query("cid == " + str(cid))['error'].iloc[0]
    rawtuxdata.iloc[i, num] = log
    i = i+1

rawtuxdata[:10]
  
#if (not (errors.query("cid == " + str(cid)).empty)):
    

,cid,date,time,vmlinux,GZIP-bzImage,GZIP-vmlinux,GZIP,BZIP2-bzImage,BZIP2-vmlinux,BZIP2,...,APDS9960,ARCH_SUPPORTS_INT128,SLABINFO,MICROCODE_AMD,ISDN_DRV_HISAX,CHARGER_BQ24190,SND_SOC_NAU8825,BH1750,NETWORK_FILESYSTEMS,cluster
700,30700,2018-06-11 12:04:27,814.281,-1,-1.0,-1,-1,-1.0,-1,-1,...,n,y,y,y,n,m,m,m,y,L2
704,30704,2018-06-11 16:28:46,318.957,-1,-1.0,-1,-1,-1.0,-1,-1,...,n,y,y,y,n,n,n,n,n,L3
717,30717,2018-06-11 16:31:52,485.380,-1,-1.0,-1,-1,-1.0,-1,-1,...,m,y,y,y,n,m,m,n,y,L3
718,30718,2018-06-11 16:31:58,228.473,-1,-1.0,-1,-1,-1.0,-1,-1,...,n,y,y,n,n,n,n,n,n,L3
722,30722,2018-06-11 16:32:37,536.941,-1,-1.0,-1,-1,-1.0,-1,-1,...,n,y,n,y,n,y,m,n,y,L3
726,30726,2018-06-11 16:34:29,282.212,-1,-1.0,-1,-1,-1.0,-1,-1,...,n,y,y,n,n,n,n,n,n,L3
730,30730,2018-06-11 16:36:31,282.096,-1,-1.0,-1,-1,-1.0,-1,-1,...,n,y,n,y,n,n,n,y,n,L4
731,30731,2018-06-11 16:36:42,315.862,-1,-1.0,-1,-1,-1.0,-1,-1,...,y,y,y,n,n,y,n,n,y,L3
737,30737,2018-06-11 16:39:28,267.500,-1,-1.0,-1,-1,-1.0,-1,-1,...,n,y,y,n,n,y,n,n,n,L3
806,30806,2018-06-11 17:04:45,266.910,-1,-1.0,-1,-1,-1.0,-1,-1,...,n,y,y,n,n,y,n,n,n,L5


In [10]:
errors = pd.read_csv('errors.csv')
clusters = []

#rawtuxdata.query("vmlinux == -1", inplace=True) 
#print (len(rawtuxdata))
#rawtuxdata['cluster'] = 'null'
#num = len(rawtuxdata.columns) - 1 
# query("vmlinux == -1").
i = 0
for row in rawtuxdata.iterrows():
    cid = row[1]['cid']
    if (row[1]['vmlinux'] != -1):
        log = 'NO_FAILURE'
    else:
        cid_error = errors.query("cid == " + str(cid))
        if (len(cid_error) > 0):
            log = cid_error['error'].iloc[0]
        else:
            print("strange case (no cid?)")
    clusters.append(log)
    #rawtuxdata.iloc[i, num] = log
#    i = i + 1
#rawtuxdata[:10]
#pd.merge(rawtuxdata, errors, on='cid', how='outer')

rawtuxdata['cluster_error'] = clusters

In [12]:
rawtuxdata.query("cluster_error != 'NO_FAILURE'")[['cid', 'cluster_error', 'vmlinux']][:10]

,cid,cluster_error,vmlinux
700,30700,L2,-1
704,30704,L3,-1
717,30717,L3,-1
718,30718,L3,-1
722,30722,L3,-1
726,30726,L3,-1
730,30730,L4,-1
731,30731,L3,-1
737,30737,L3,-1
806,30806,L5,-1


In [33]:
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'")['cluster_error'].unique()

array(['L3', 'L5', 'L12', 'L8', 'L19', 'L22', 'L16', 'L24', 'L25', 'L27',
       'L30', 'L31', 'L1', 'L33', 'L26', 'L40', 'L10', 'L32', 'L44',
       'L37', 'L47', 'L48', 'L42', 'L50', 'L51', 'L41', 'L46', 'L57',
       'L54', 'L59', 'L60'], dtype=object)

In [29]:
#rawtuxdata.cluster_error.count()
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'").groupby('cluster_error').count()[['cid']]

,cid
cluster_error,
L1,5
L10,1
L12,4
L16,1
L19,1
L22,1
L24,1149
L25,4
L26,16


In [45]:
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y' & cluster_error == 'L24'").groupby(['AIC79XX_BUILD_FIRMWARE', 'WANXL_BUILD_FIRMWARE', 'GENERIC_ALLOCATOR', 'DRM_VBOXVIDEO']).count()[['cid']]

#in 10106 of 1149 errors the option GENERIC_ALLOCATOR is also selected


cid
AIC79XX_BUILD_FIRMWARE WANXL_BUILD_FIRMWARE GENERIC_ALLOCATOR DRM_VBOXVIDEO     
n                      n                    n                 m                4
                                                              n               24
                                                              y                4
                                            y                 m               68
                                                              n              556
                                                              y              122
y                      n                    n                 n               10
                                                              y                1
                                            y                 m               35
                                                              n              270
                                                              y               55

In [46]:
rawtuxdata.query("GENERIC_ALLOCATOR == 'y'").groupby('cluster_error').count()[['cid']]

,cid
cluster_error,
L1,163
L10,125
L11,7
L12,38
L13,1
L14,2
L15,5
L16,2
L17,1


In [16]:
rawtuxdata.query("AIC7XXX_BUILD_FIRMWARE == 'y'")['cid'].unique()

array([30718, 30722, 30731, ..., 85954, 85977, 86083])

In [107]:
from pandas import DataFrame
import MySQLdb
import bz2

cids = []
errors = []
       
try:
    socket = MySQLdb.connect("148.60.11.195", "script", "irmadb2019", "IrmaDB_prod")
    cursor = socket.cursor()
    cursor.execute("SELECT cid, errlog_file FROM Compilations WHERE core_size=-1 and cid > 30000 and cid < 100000")
    #cursor.execute("SELECT cid, errlog_file FROM Compilations WHERE cid in (35706, 57116, 33574, 56926, 44782, 78452, 37565, 65481, 77080, 60274)")
    
    for row in cursor.fetchall():
        cid = row[0]   
        if (row[1] is None):
           print ("Unable to retrieve cid=", str(cid))
        try:
           log_error = bz2.decompress(row[1]).decode('utf-8')
           #log_error = log_error.replace('\r', '').replace('\n', '')
        except Exception as e:
           print(str(e),"\n" + "Unable to decompress... ", file=sys.stderr)
           exit(-1)
        
        cids.append (cid)
        errors.append (log_error)
    
    Errlogs = {'cid': cids, 'error_message': errors}
    df = DataFrame(Errlogs, columns= ['cid', 'error_message'])
    export_csv = df.to_csv ('errlogs.csv', index = None, header=True) 
    
except Exception as e:
    print(str(e),"\n" + "Unable to connect to database c = " + str(cid), file=sys.stderr)
    exit(-1)    
finally:
    cursor.close()
    socket.close()

End!


In [89]:
df['cid']

0    33574
1    35706
2    37565
3    44782
4    56926
5    57116
6    60274
7    65481
8    77080
9    78452
Name: cid, dtype: int64

In [110]:
df.query("cid == '57116'")['error_message']

1606    Cyclomatic Complexity 1 kernel/bounds.c:foo\nC...
Name: error_message, dtype: object

In [111]:
len(df)

3538